## Revision History

In [97]:
# Change_date         revision_number     change_description                           author
# 05/09/2024          1                   initial check-in                             Kranthi
#1. prepare normalized tables
#2. generate unique id for each row- event_name+user_pseudo_id+event_ts+event_bundle_sequence_id
#3. CCONCAT(event_name, "-", user_pseudo_id, "-", CAST(event_timestamp AS STRING), "-", CAST(event_bundle_sequence_id AS STRING)) AS event_id
#4. event_params, user_properties, items, item_params
#5. is_sku attributes at item level 

In [1]:
%run /utils/common_functions

In [ ]:
%%sql
create schema if not exists silver  

## print metadata of a struct

In [ ]:
# df_item_params = spark.sql("""select * from (select distinct key, value, row_number() over(partition by key order by value) rn
#  from (select explode(col.item_params) from (select explode(items) from raw.ga4_events))
# where key IN ('isFinalSale','product_color','size_color_style_id','product_size','sku','product_on_sale_sale_type'))
# where rn =1 limit 100""")

# for j in df_item_params.collect():
#      #print(j[0])
#      if j[1].string_value is not None: 
#        val1 =  'string_value'
#      elif j[1].int_value is not None:
#        val1 =  'int_value'
#      elif j[1].float_value is not None:  
#        val1 =  'float_value'   
#      elif j[1].double_value is not None:  
#        val1 =  'double_value'
#      print(",'event_params."+j[0]+'.'+val1+"':'event_params_"+j[0]+"'")   

df = spark.sql('select * from raw.ga4_events')
#df.schema["ecommerce"].dataType.elementType.fieldNames()
#print(df.schema["ecommerce"].dataType.fieldNames())
for j in df.schema["items"].dataType.elementType.fieldNames():
    print(j)
    #print(",'"+'items.'+j+"':'"+'items_'+j+"'")     


In [ ]:
items1 = df.selectExpr('event_params').collect()[0][0].items()

# print(items1)
# for j in items1:
#     #print(j[0])
#     if j[1].string_value is not None: 
#       val1 =  'string_value'
#     elif j[1].int_value is not None:
#       val1 =  'int_value'
#     elif j[1].float_value is not None:  
#       val1 =  'float_value'   
#     elif j[1].double_value is not None:  
#       val1 =  'double_value'
#     print(",'event_params."+j[0]+'.'+val1+"':'event_params_"+j[0]+"'")    

In [ ]:
df = spark.sql('select * from raw.ga4_events')
#df.schema["ecommerce"].dataType.elementType.fieldNames()
#print(df.schema["ecommerce"].dataType.fieldNames())
for j in df.schema["items"].dataType.elementType.fieldNames():
    #print('device_'+j)
    print(",'"+'items.'+j+"':'"+'items_'+j+"'")
#display(df)
#items1 = df.selectExpr('event_params').collect()[0][0].items()

# print(items1)
# for j in items1:
#     #print(j[0])
#     if j[1].string_value is not None: 
#       val1 =  'string_value'
#     elif j[1].int_value is not None:
#       val1 =  'int_value'
#     elif j[1].float_value is not None:  
#       val1 =  'float_value'   
#     elif j[1].double_value is not None:  
#       val1 =  'double_value'
#     print(",'event_params."+j[0]+'.'+val1+"':'event_params_"+j[0]+"'")     

In [ ]:
ga4_schema = StructType([
    StructField("event_date", StringType(), True),
    StructField("event_timestamp", LongType(), True),
    StructField("event_name", StringType(), True),
    StructField("event_params", MapType(StringType(), StructType([
        StructField("string_value", StringType(), True),
        StructField("int_value", LongType(), True),
        StructField("float_value", DoubleType(), True),
        StructField("double_value", DoubleType(), True)
    ])), True),
    StructField("event_previous_timestamp", LongType(), True),
    StructField("event_value_in_usd", DoubleType(), True),
    StructField("event_bundle_sequence_id", LongType(), True),
    StructField("event_server_timestamp_offset", LongType(), True),
    StructField("user_id", StringType(), True),
    StructField("user_pseudo_id", StringType(), True),
    StructField("privacy_info", StructType([
        StructField("analytics_storage", StringType(), True),
        StructField("ads_storage", StringType(), True),
        StructField("uses_transient_token", StringType(), True)
    ]), True),
    StructField("user_properties", MapType(StringType(), StructType([
        StructField("string_value", StringType(), True),
        StructField("int_value", LongType(), True),
        StructField("float_value", DoubleType(), True),
        StructField("double_value", DoubleType(), True),
        StructField("set_timestamp_micros", LongType(), True)
    ])), True),
    StructField("user_first_touch_timestamp", LongType(), True),
    StructField("user_ltv", StructType([
        StructField("revenue", DoubleType(), True),
        StructField("currency", StringType(), True)
    ]), True),
    StructField("device", StructType([
        StructField("category", StringType(), True),
        StructField("mobile_brand_name", StringType(), True),
        StructField("mobile_model_name", StringType(), True),
        StructField("mobile_marketing_name", StringType(), True),
        StructField("mobile_os_hardware_model", StringType(), True),
        StructField("operating_system", StringType(), True),
        StructField("operating_system_version", StringType(), True),
        StructField("vendor_id", StringType(), True),
        StructField("advertising_id", StringType(), True),
        StructField("language", StringType(), True),
        StructField("is_limited_ad_tracking", StringType(), True),
        StructField("time_zone_offset_seconds", LongType(), True),
        StructField("browser", StringType(), True),
        StructField("browser_version", StringType(), True),
        StructField("web_info", StructType([
            StructField("browser", StringType(), True),
            StructField("browser_version", StringType(), True),
            StructField("hostname", StringType(), True)
        ]), True)
    ]), True),
    StructField("geo", StructType([
        StructField("city", StringType(), True),
        StructField("country", StringType(), True),        
        StructField("continent", StringType(), True),
        StructField("region", StringType(), True),        
        StructField("sub_continent", StringType(), True),
        StructField("metro", StringType(), True)
    ]), True),
    StructField("app_info", StructType([
        StructField("id", StringType(), True),
        StructField("version", StringType(), True),
        StructField("install_store", StringType(), True),
        StructField("firebase_app_id", StringType(), True),
        StructField("install_source", StringType(), True)
    ]), True),
    StructField("traffic_source", StructType([
        StructField("name", StringType(), True),
        StructField("medium", StringType(), True),
        StructField("source", StringType(), True)
    ]), True),
    StructField("stream_id", StringType(), True),
    StructField("platform", StringType(), True),
    StructField("event_dimensions", StructType([
        StructField("hostname", StringType(), True)
    ]), True),
    StructField("ecommerce", StructType([
        StructField("total_item_quantity", LongType(), True),
        StructField("purchase_revenue_in_usd", DoubleType(), True),
        StructField("purchase_revenue", DoubleType(), True),
        StructField("refund_value_in_usd", DoubleType(), True),
        StructField("refund_value", DoubleType(), True),
        StructField("shipping_value_in_usd", DoubleType(), True),
        StructField("shipping_value", DoubleType(), True),
        StructField("tax_value_in_usd", DoubleType(), True),
        StructField("tax_value", DoubleType(), True),
        StructField("unique_items", LongType(), True),
        StructField("transaction_id", StringType(), True),
        StructField("items", ArrayType(StructType([
            StructField("item_id", StringType(), True),
            StructField("item_name", StringType(), True),
            StructField("item_brand", StringType(), True),
            StructField("item_variant", StringType(), True),
            StructField("item_category", StringType(), True),
            StructField("item_category2", StringType(), True),
            StructField("item_category3", StringType(), True),
            StructField("item_category4", StringType(), True),
            StructField("item_category5", StringType(), True),
            StructField("price_in_usd", DoubleType(), True),
            StructField("price", DoubleType(), True),
            StructField("quantity", LongType(), True),
            StructField("item_revenue_in_usd", DoubleType(), True),
            StructField("item_revenue", DoubleType(), True),
            StructField("item_refund_in_usd", DoubleType(), True),
            StructField("item_refund", DoubleType(), True),
            StructField("coupon", StringType(), True),
            StructField("affiliation", StringType(), True),
            StructField("location_id", StringType(), True),
            StructField("item_list_id", StringType(), True),
            StructField("item_list_name", StringType(), True),
            StructField("item_list_index", StringType(), True),
            StructField("promotion_id", StringType(), True),
            StructField("promotion_name", StringType(), True),
            StructField("creative_name", StringType(), True),
            StructField("creative_slot", StringType(), True),
            StructField("item_params", MapType(StringType(), StructType([
                StructField("string_value", StringType(), True),
                StructField("int_value", LongType(), True),
                StructField("float_value", DoubleType(), True),
                StructField("double_value", DoubleType(), True)
            ])), True)
        ])), True)
    ]), True),
    StructField("collected_traffic_source", StructType([
        StructField("manual_campaign_id", StringType(), True),
        StructField("manual_campaign_name", StringType(), True),
        StructField("manual_source", StringType(), True),
        StructField("manual_medium", StringType(), True),
        StructField("manual_term", StringType(), True),
        StructField("manual_content", StringType(), True),
        StructField("gclid", StringType(), True),
        StructField("dclid", StringType(), True),
        StructField("srsltid", StringType(), True)
    ]), True),
    StructField("is_active_user", BooleanType(), True),
    StructField("property_brand", StringType(), True),
    StructField('brand_country', StringType(), True),
    StructField('lineage_source',StringType(), True)
   # 
    
])

In [6]:



ga4_events_cols1 = {
'event_date':'event_date'
,'property_brand':'property_brand'
,'brand_country':'brand_country'
,'lineage_source':'lineage_source'
,'event_name':'event_name'
,'user_pseudo_id':'user_pseudo_id'
,'event_timestamp':'event_timestamp'
,'event_bundle_sequence_id':'event_bundle_sequence_id'
,"""sha(concat(event_name,'-', user_pseudo_id,'-', CAST(event_timestamp AS STRING),'-', row_number()over(partition by event_name, user_pseudo_id,event_timestamp order by event_timestamp)))""":'unique_key'
,'user_id':'user_id'
,'event_previous_timestamp':'event_previous_timestamp'
,'event_value_in_usd':'event_value_in_usd'
,'event_server_timestamp_offset':'event_server_timestamp_offset'
,'privacy_info.uses_transient_token':'privacy_info_uses_transient_token' ##### RECORD
,'privacy_info.ads_storage':'privacy_info_ads_storage'
,'privacy_info.analytics_storage':'privacy_info_analytics_storage'
,'user_first_touch_timestamp':'user_first_touch_timestamp'
,'user_ltv.revenue':'user_ltv_revenue' ##RECORD
,'user_ltv.currency':'user_ltv_currency' ##RECORD
,'device.category':'device_category'
,'device.mobile_brand_name':'device_mobile_brand_name'
,'device.mobile_model_name':'device_mobile_model_name'
,'device.mobile_marketing_name':'device_mobile_marketing_name'
,'device.mobile_os_hardware_model':'device_mobile_os_hardware_model'
,'device.operating_system':'device_operating_system'
,'device.operating_system_version':'device_operating_system_version'
,'device.vendor_id':'device_vendor_id'
,'device.advertising_id':'device_advertising_id'
,'device.language':'device_language'
,'device.is_limited_ad_tracking':'device_is_limited_ad_tracking'
,'device.time_zone_offset_seconds':'device_time_zone_offset_seconds'
,'device.browser':'device_browser'
,'device.browser_version':'device_browser_version'
,'device.web_info.browser':'device_web_info_browser'
,'device.web_info.browser_version':'device_web_info_browser_version'
,'device.web_info.hostname':'device_web_info_hostname'
,'geo.continent': 'geo_continent'
,'geo.country': 'geo_country'
,'geo.city':'geo_city' ##RECORD map, array, struct
,'geo.region': 'geo_region'
,'geo.sub_continent': 'geo_sub_continent'
,'geo.metro': 'geo_metro'

,'app_info.id':'app_info_id'
,'app_info.version':'app_info_version'
,'app_info.install_store':'app_info_install_store'
,'app_info.firebase_app_id':'app_info_firebase_app_id'
,'app_info.install_source':'app_info_install_source'
,'traffic_source.name':'traffic_source_name' ##RECORD
,'traffic_source.medium':'traffic_source_medium'
,'traffic_source.source':'traffic_source_source'
,'stream_id':'stream_id'
,'platform':'platform'
,'event_dimensions.hostname':'event_dimensions_hostname' ##RECORD
,'ecommerce.total_item_quantity':'ecommerce_total_item_quantity'
,'ecommerce.purchase_revenue_in_usd':'ecommerce_purchase_revenue_in_usd'
,'ecommerce.purchase_revenue':'ecommerce_purchase_revenue'
,'ecommerce.refund_value_in_usd':'ecommerce_refund_value_in_usd'
,'ecommerce.refund_value':'ecommerce_refund_value'
,'ecommerce.shipping_value_in_usd':'ecommerce_shipping_value_in_usd'
,'ecommerce.shipping_value':'ecommerce_shipping_value'
,'ecommerce.tax_value_in_usd':'ecommerce_tax_value_in_usd'
,'ecommerce.tax_value':'ecommerce_tax_value'
,'ecommerce.unique_items':'ecommerce_unique_items'
,'ecommerce.transaction_id':'ecommerce_transaction_id'

,'collected_traffic_source.manual_campaign_id':'collected_traffic_source_manual_campaign_id'
,'collected_traffic_source.manual_campaign_name':'collected_traffic_source_manual_campaign_name'
,'collected_traffic_source.manual_source':'collected_traffic_source_manual_source'
,'collected_traffic_source.manual_medium':'collected_traffic_source_manual_medium'
,'collected_traffic_source.manual_term':'collected_traffic_source_manual_term'
,'collected_traffic_source.manual_content':'collected_traffic_source_manual_content'
,'collected_traffic_source.gclid':'collected_traffic_source_gclid'
,'collected_traffic_source.dclid':'collected_traffic_source_dclid'
,'collected_traffic_source.srsltid':'collected_traffic_source_srsltid'
,'is_active_user':'is_active_user'}



ga4_events_base_cols =  [i+' as '+ j for i,j in ga4_events_cols1.items()]
gua_events_base_cols =  ga4_events_cols1
## GUA does not have device_browser and device.browser_version replace with null 
gua_events_base_cols['NULL']=gua_events_base_cols.pop("device.browser")
gua_events_base_cols['NULL1']=gua_events_base_cols.pop("device.browser_version")
#print(gua_events_base_cols)
df_ga4 = spark.sql('select * from raw.ga4_events')
#df_gua = spark.sql('select * from raw.gua_events')
#df_ga4 = df_ga4.selectExpr(ga4_events_base_cols)
#df_gua = df_gua.selectExpr(gua_events_base_cols)
#df_ga4.selectExpr(ga4_events_base_cols).createOrReplaceTempView('ga4_events_base_tab')
#df = df_gua4.unionAll(df_gua)
#display(df)
df_ga4.selectExpr(ga4_events_base_cols)\
    .repartition('event_date','property_brand','brand_country')\
    .write.format("delta")\
    .mode("overwrite")\
    .option("path",f"{gold_adls_path}GA4/events")\
    .option("mergeSchema", "true")\
    .partitionBy('event_date','property_brand','brand_country')\
    .saveAsTable('lakedb_gold.ga4_events_base_table')




    

In [5]:
%%sql
select count(*) from delta.`abfss://raw@azwwwprodprdadapadls.dfs.core.windows.net/SAP/BW/ZSHPRPCD$P`

In [ ]:
%%sql
--select * from raw.ga4_events
select * from lakedb_gold.ga4_events_base_table

In [4]:
# events_cols3 = {'event_date':'event_date'
# ,'property_brand':'property_brand'
# ,'brand_country':'brand_country'
# ,'lineage_source':'lineage_source'
# ,'event_name':'event_name'
# ,'user_pseudo_id':'user_pseudo_id'
# ,'event_timestamp':'event_timestamp'
# ,'unique_key':'unique_key'
# #,"""sha(concat(event_name,'-', user_pseudo_id,'-', CAST(event_timestamp AS STRING),'-', row_number()over(partition by event_name, user_pseudo_id,event_timestamp order by event_timestamp)))""":'unique_key'

# ,'user_properties.customer_service_order.string_value':'uprop_customer_service_order'
# ,'user_properties.client_id.string_value':'uprop_client_id'
# ,'user_properties.site_id.string_value':'uprop_site_id'
# ,'user_properties.user_id.string_value':'uprop_user_id_str'
# ,'user_properties.user_id.int_value':'uprop_user_id_int'
# ,'user_properties.user_id_up.string_value':'uprop_user_id_up_str'
# ,'user_properties.user_id_up.int_value':'uprop_user_id_up_int'
# ,'user_properties.customer_service_order.set_timestamp_micros':'uprop_customer_service_order_ts'
# ,'user_properties.client_id.set_timestamp_micros':'uprop_client_id_ts'
# ,'user_properties.site_id.set_timestamp_micros':'uprop_site_id_ts'
# ,'user_properties.user_id.set_timestamp_micros':'uprop_user_id_ts'
# ,'user_properties.user_id_up.set_timestamp_micros':'uprop_user_id_up_ts'
# ,'user_properties.prevenue_28d.set_timestamp_micros':'uprop_prevenue_28d_ts'}

df_user_prop = spark.sql("""select event_date
,property_brand
,brand_country
,lineage_source
,event_name
,user_pseudo_id
,event_timestamp
,unique_key
,user_properties.customer_service_order.string_value as uprop_customer_service_order
,user_properties.client_id.string_value as uprop_client_id
,user_properties.site_id.string_value as uprop_site_id
,user_properties.user_id.string_value as uprop_user_id_str
,user_properties.user_id.int_value as uprop_user_id_int
,user_properties.user_id_up.string_value as uprop_user_id_up_str
,user_properties.user_id_up.int_value as uprop_user_id_up_int
,user_properties.customer_service_order.set_timestamp_micros as uprop_customer_service_order_ts
,user_properties.client_id.set_timestamp_micros as uprop_client_id_ts
,user_properties.site_id.set_timestamp_micros as uprop_site_id_ts
,user_properties.user_id.set_timestamp_micros as uprop_user_id_ts
,user_properties.user_id_up.set_timestamp_micros as uprop_user_id_up_ts
,user_properties.prevenue_28d.set_timestamp_micros as uprop_prevenue_28d_ts
from (select event_date
,property_brand
,brand_country
,lineage_source
,event_name
,user_pseudo_id
,event_timestamp
,sha(concat(event_name,'-', user_pseudo_id,'-', CAST(event_timestamp AS STRING),'-', row_number()over(partition by event_name, user_pseudo_id,event_timestamp order by event_timestamp))) as unique_key
,user_properties
from raw.ga4_events)""")
#events_cols3 =  [i+' as '+ j for i,j in events_cols3.items()]
df_user_prop.selectExpr('*')\
    .repartition('event_date','property_brand','brand_country')\
    .write.format("delta")\
    .mode("overwrite")\
    .option("path",f"{gold_adls_path}GA4/event_user_properties")\
    .option("mergeSchema", "true")\
    .partitionBy('event_date','property_brand','brand_country')\
    .saveAsTable('lakedb_gold.ga4_event_user_properties')


In [5]:
%%sql
select * from lakedb_gold.ga4_event_user_properties

In [ ]:
%%sql
-- select user_pseudo_id,  event_name, event_timestamp ,event_bundle_sequence_id , count(*)
-- from ga4_events_base_table
-- group by user_pseudo_id,  event_name, event_timestamp ,event_bundle_sequence_id
-- having count(*)>1;

select * from  ga4_events_base_table 
where user_pseudo_id = 40981350.1714402206
and event_name = 'product_interaction'
and event_timestamp = '1714658559052625'
and event_bundle_sequence_id = 240307025

In [ ]:
df = spark.sql('select distinct key, value.* from(select explode(event_params) from raw.ga4_events)a' )
display(df)

In [10]:
df = spark.sql(f"""select event_date
,property_brand
,brand_country
,lineage_source
,event_name
,user_pseudo_id
,event_timestamp
,unique_key
,explode(event_params) from (select event_date
,property_brand
,brand_country
,lineage_source
,event_name
,user_pseudo_id
,event_timestamp
,event_params
,sha(concat(event_name,'-', user_pseudo_id,'-', CAST(event_timestamp AS STRING),'-', row_number()over(partition by event_name, user_pseudo_id,event_timestamp order by event_timestamp))) as unique_key

from raw.ga4_events ) """)
df.repartition('event_date','property_brand','brand_country')\
    .write.format("delta")\
    .mode("overwrite")\
    .option("path",f"{gold_adls_path}GA4/event_parameters")\
    .option("mergeSchema", "true")\
    .partitionBy('event_date','property_brand','brand_country')\
    .saveAsTable('lakedb_gold.ga4_event_parameters')


In [ ]:
items_cols = {'event_date':'event_date'
,'property_brand':'property_brand'
,'brand_country':'brand_country'
,'lineage_source':'lineage_source'
,'event_name':'event_name'
,'user_pseudo_id':'user_pseudo_id'
,'event_timestamp':'event_timestamp'
,'unique_key':'unique_key'
#,"""sha(concat(event_name,'-', user_pseudo_id,'-', CAST(event_timestamp AS STRING),'-', row_number()over(partition by event_name, user_pseudo_id,event_timestamp order by event_timestamp)))""":'unique_key'
,'items.item_id':'items_item_id'
,'items.item_name':'items_item_name'
,'items.item_brand':'items_item_brand'
,'items.item_variant':'items_item_variant'
,'items.item_category':'items_item_category'
,'items.item_category2':'items_item_category2'
,'items.item_category3':'items_item_category3'
,'items.item_category4':'items_item_category4'
,'items.item_category5':'items_item_category5'
,'items.price_in_usd':'items_price_in_usd'
,'items.price':'items_price'
,'items.quantity':'items_quantity'
,'items.item_revenue_in_usd':'items_item_revenue_in_usd'
,'items.item_revenue':'items_item_revenue'
,'items.item_refund_in_usd':'items_item_refund_in_usd'
,'items.item_refund':'items_item_refund'
,'items.coupon':'items_coupon'
,'items.affiliation':'items_affiliation'
,'items.location_id':'items_location_id'
,'items.item_list_id':'items_item_list_id'
,'items.item_list_name':'items_item_list_name'
,'items.item_list_index':'items_item_list_index'
,'items.promotion_id':'items_promotion_id'
,'items.promotion_name':'items_promotion_name'
,'items.creative_name':'items_creative_name'
,'items.creative_slot':'items_creative_slot'
,'items.item_params.isFinalSale.string_value':'item_params_isFinalSale'
,'items.item_params.product_color.int_value':'item_params_product_color'
,'items.item_params.size_color_style_id.double_value':'item_params_size_color_style_id'
,'items.item_params.product_size.double_value':'item_params_product_size'
,'items.item_params.sku.int_value':'item_params_sku'
,'items.item_params.product_on_sale_sale_type.string_value':'item_params_product_on_sale_sale_type'
}

df = spark.sql("""select  event_date
,property_brand
,brand_country
,lineage_source
,event_name
,user_pseudo_id
,event_timestamp
,unique_key
,explode(items) as items
 from (select event_date
,property_brand
,brand_country
,lineage_source
,event_name
,user_pseudo_id
,event_timestamp
,sha(concat(event_name,'-', user_pseudo_id,'-', CAST(event_timestamp AS STRING),'-', row_number()over(partition by event_name, user_pseudo_id,event_timestamp order by event_timestamp))) as unique_key
 from raw.ga4_events)""")
item_col_exp = [i + ' as '+ j for (i,j) in items_cols.items()]
df = df.selectExpr(item_col_exp)
display(df)
df.repartition('event_date','property_brand','brand_country')\
    .write.format("delta")\
    .mode("overwrite")\
    .option("path",f"{gold_adls_path}GA4/item_data")\
    .option("mergeSchema", "true")\
    .partitionBy('event_date','property_brand','brand_country')\
    .saveAsTable('lakedb_gold.ga4_item_data')

In [ ]:
%%sql
select * from silver.ga4_item_data 
where 1=1
--and item_params_product_color is not null
--and item_params_product_color is not null
--select items.item_params.isFinalSale.string_value from (select explode(items) as items from raw.ga4_events) 
--select explode(items) as items from raw.ga4_events

In [ ]:
%%sql
select items.item_id from raw.ga4_events 
where items.item_id is not null and length(items.item_id)>0;
-- select items.item_params.isFinalSale.string_value
-- ,items.item_params.product_color.int_value
-- ,items.item_params.size_color_style_id.double_value
-- ,items.item_params.product_size.double_value
-- ,items.item_params.sku.int_value
-- ,items.item_params.product_on_sale_sale_type.string_value 
-- from raw.ga4_events where items.item_params is not null

In [36]:
# df = spark.sql("""select  user_properties.site_id.string_value, user_properties.site_id.set_timestamp_micros
# from raw.ga4_events""")
df = spark.sql("""select user_properties.user_id.string_value, user_properties.user_id.int_value, user_properties.user_id.set_timestamp_micros
from raw.ga4_events where user_properties.user_id.string_value is not null""")
display(df)

In [49]:
df = spark.sql(f"""select  explode(user_properties)
from raw.ga4_events limit 10""")
display(df)
# items1 = df.collect()[0][0].items()
# for j in items1:
#     #print(j[0])
#     if j[1].string_value is not None: 
#       val1 =  'string_value'
#     elif j[1].int_value is not None:
#       val1 =  'int_value'
#     elif j[1].float_value is not None:  
#       val1 =  'float_value'   
#     elif j[1].double_value is not None:  
#       val1 =  'double_value'
#     elif j[1].set_timestamp_micros is not None:
#       val1 = 'long_value'    
#     print(",'user_properties."+j[0]+'.'+val1+"':'user_prop_"+j[0]+"'")  

In [10]:
%%sql
select count(*) from raw.gua_events;

In [5]:
%%sql
select explode(event_params) 
from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events/`
-- select event_params.session_engaged.string_value 
-- from delta.`abfss://raw@azwwwnonproddevadapadls.dfs.core.windows.net/GA4/events/`

## Calculate metrics sessions

In [ ]:
%%sql

SELECT
  device.deviceCategory AS device_category,
  count(distinct concat(user_pseudo_id,(select value.int_value from unnest(event_params) where key = 'ga_session_id'))) AS session_count,
  COUNTIF(event_name = 'add_to_cart') AS adds_to_cart
  COUNTIF(event_name IN ('purchase', 'transaction')) AS orders
  SUM(IF(event_name = 'purchase', items.quantity, 0)) AS quantity
  COUNTIF(event_name = 'purchase') AS Item_sales
FROM
 raw.ga4_events
GROUP BY
               device_category

In [101]:
%%sql
select a.device_category, count(b.items_item_id) 
from lakedb_gold.ga4_events_base_table a left join silver.ga4_item_data b
on a.unique_key = b.unique_key
where a.event_date between '20240421' and '20240427' 
group by a.device_category

In [126]:
%%sql
with t1 as (select concat(event_name,'-'
    , user_pseudo_id,'-'
        , CAST(event_timestamp AS STRING)
        ,'-', row_number()over(partition by event_name, user_pseudo_id,event_timestamp order by event_timestamp)) as event_id
 from lakedb_gold.ga4_events_base_table
 where event_name = 'internal_link_click'
    and user_pseudo_id = '598041881.1683471910'
        and event_timestamp = '1683504759610892')
    
        select count(*) from (select event_id, *  from t1, lakedb_gold.ga4_events_base_table
 where event_name = 'internal_link_click'
    and user_pseudo_id = '598041881.1683471910'
        and event_timestamp = '1683504759610892')


In [147]:
%%sql
with t1 as (
select device_category,'sessions' as metric
, count( distinct concat(a.user_pseudo_id,a.value.int_value)) as cnt
 from (select device.category as device_category
 , user_pseudo_id
 , explode(event_params)
                from raw.ga4_events a
                where event_date between '20240421' and '20240427') a
                where a.key = 'ga_session_id' 
  group by device_category) 
  , t2 as (select device.category,'carts',COUNT_IF(event_name = 'add_to_cart') AS adds_to_cart
  --COUNT_IF(event_name IN ('purchase', 'transaction')) AS orders 
  from  raw.ga4_events a
                where event_date between '20240421' and '20240427'             
  group by device.category)
  ,t3 as (select device.category, 'orders',COUNT_IF(event_name IN ('purchase', 'transaction'))
from  raw.ga4_events a
                where event_date between '20240421' and '20240427'             
  group by device.category)
 , t5 as (select device_category, 'quantity',SUM(items.quantity) 
        from (select device.category as device_category, event_name, explode(items) as items
       from raw.ga4_events where event_name = 'purchase'
        and event_date between '20240421' and '20240427'   ) group by device_category
        )
  ,t4 as(select * from t1 
  union all
  select * from t2 
  union all
  select * from t3
  union all 
  select * from t5)
  SELECT *
 FROM t4
 PIVOT (
   sum(cnt)
   FOR metric IN ('sessions','carts','orders','quantity')
)

In [146]:
%%sql
select device_category, 'quantity',SUM(items.quantity) 
        from (select device.category as device_category, event_name, explode(items) as items
       from raw.ga4_events where event_name = 'purchase') group by device_category

In [112]:
%%sql
with visits as (select device_category, count( distinct concat(b.user_pseudo_id,b.value.int_value))
 from lakedb_gold.ga4_events_base_table a inner join silver.ga4_event_parameters b
 on a.event_date = b.event_date and a.unique_key = b.unique_key
where b.key = 'ga_session_id' 
    and b.event_date between '20240421' and '20240427' 
        group by device_category )
     select * from visits;
--select * from silver.ga4_event_parameters
--where event_date between '2024-04-21' and '2024-04-27' 
  --  and key = 'ga_session_id'

In [ ]:
%%sql
SELECT
  device.category AS device_category
 -- ,count_if(distinct concat(user_pseudo_id,event_params.value.int_value from unnest(event_params) where key = 'ga_session_id'))) AS session_count
  ,COUNT_IF(event_name = 'add_to_cart') AS adds_to_cart
  ,COUNT_IF(event_name IN ('purchase', 'transaction')) AS orders
 -- ,SUM(case when event_name = 'purchase' then items.quantity else [0] end) AS quantity
  ,COUNT_IF(event_name = 'purchase') AS Item_sales
FROM
 raw.ga4_events
 where event_date between 20240421 and 20240427
GROUP BY  device.category;



